## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / |\
     /  |  \
 正面论述| 反面论述
     \  | /
      \ |/
     最终总结
```

In [1]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出评价后，生成最终回应"),
        ]
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [1]:
# by wy：优化代码 之一
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI
from operator import itemgetter
from typing import Any

# 定义一个共用的模型实例
model = ChatOpenAI(model="gpt-4o-mini")

# 定义一个通用的论点生成模板
arg_prompt = "列出关于{base_response}的{aspect}方面"
argument_template = ChatPromptTemplate.from_template(arg_prompt)

# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | model
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}  # by wy：将输出的str作为字典中一个键的值
)

# 创建正面和反面论证的处理链
arguments_for = argument_template.partial(aspect="正面或有利的") | model | StrOutputParser()
arguments_against = argument_template.partial(aspect="反面或不利的") | model | StrOutputParser()

# 创建最终响应者
messages = [
    ("ai", "{original_response}"),
    ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
    ("system", "给出评价后，生成最终回应"),
]
final_responder = (
    ChatPromptTemplate.from_messages(messages)
    | model
    | StrOutputParser()
)

# 构建完整的处理链


def create_chain() -> Any:
    return (
        planner
        | RunnableParallel(
            {
                "results_1": arguments_for,
                "results_2": arguments_against,
                "original_response": itemgetter("base_response"),
            }
        )
        | final_responder
    )


# 使用类型注解
chain: Any = create_chain()

In [2]:
# by wy：优化代码 之二
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI
from operator import itemgetter

model = ChatOpenAI(model="gpt-4o-mini")

# 构造初始化链
start_chain = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点：{input}")
    | model
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()} # by wy：将输出的str作为字典中一个键的值
)

# 构造最终链
final_system_tpl = "给出评价后，生成最终回应"
final_ai_tpl = "原始论点：\n{original_response}"
final_human_tpl = "正面观点：\n{result_for} \n\n 反面观点：\n{result_against}"
final_msgs = [
    SystemMessagePromptTemplate.from_template(final_system_tpl),
    AIMessagePromptTemplate.from_template(final_ai_tpl),
    HumanMessagePromptTemplate.from_template(final_human_tpl)
]

final_chain = (
    ChatPromptTemplate.from_messages(final_msgs)
    | model
    | StrOutputParser()
)

# 构造中间链
arg_prompt = "列出关于{base_response}的{aspect}方面"
arg_tpl = ChatPromptTemplate.from_template(arg_prompt)
arg_for = arg_tpl.partial(aspect="正面或有利的") | model | StrOutputParser()
arg_against = arg_tpl.partial(aspect="反面或不利的") | model | StrOutputParser()

middle_chain = RunnableParallel(
    result_for=arg_for,
    result_against=arg_against,
    original_response=itemgetter("base_response")
)

# 构造完整链
chain = start_chain | middle_chain | final_chain
chain_invoke_args = {"input": "读仙侠小说的意义"}

# 非流式输出
# chain.invoke(chain_invoke_args)

# 流式输出
for s in chain.stream(chain_invoke_args):
    print(s, end="", flush=True)

关于仙侠小说的讨论确实需要从多个角度进行全面的分析。正面观点指出，仙侠小说不仅能够传承文化、激发想象力、提供人生哲理，还能为读者提供心理上的逃避与成长的机会。这些因素无疑为读者的个人发展和文化认同提供了积极的支持。

然而，反面观点也提醒我们，沉迷于仙侠小说可能导致脱离现实、价值观偏差等问题。尤其是在现代社会中，读者需要合理安排时间，保持对现实的关注，以免影响生活的平衡与人际关系的发展。

综上所述，仙侠小说的阅读体验既有其独特的魅力和深远的意义，也隐含着需警惕的潜在问题。读者在享受这些作品时，应该保持理性，积极吸取其中的正面价值观，同时警惕可能带来的负面影响，以实现更全面的个人成长与发展。

In [2]:
chain.invoke({"input": "网红对社会的价值"})

'评价：\n\n网红在现代社会中确实发挥着多重角色，其正面影响和反面问题并存。正面观点中强调的知识分享、文化传播和商业价值的推动，展示了网红在促进信息流通和多样性方面的重要性。尤其是在社会议题的关注和社群建设上，网红能够有效地引导公众讨论，提升社会意识。\n\n然而，反面观点中也指出了网红可能带来的信息可信度低、偶像崇拜、消费主义等问题，这些问题不可忽视。尤其是对于年轻人来说，过度的偶像崇拜和物质文化的追求可能影响其心理健康和价值观的形成。此外，隐私问题和内容同质化也是当前网红文化中的重要考量。\n\n最终回应：\n\n综上所述，网红在当今社会中无疑是一个复杂而多元的现象。我们需要充分认识到他们在信息传播、文化交流和商业价值等方面的积极作用，同时也要警惕由此带来的潜在负面影响。在享受网红带来的便利和乐趣时，保持批判性思维和理性判断尤为重要。通过合理利用网红的影响力，促进社会的积极发展，并在个人成长中形成健康的价值观与消费观，才能在这个快速变化的时代中找到平衡。'

#### 流式输出

In [3]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "网红对社会的价值"}):
    print(s, end="", flush=True)

评价：

在现代社会中，网红的影响力呈现出多维度的特征，既有显著的正面效果，也伴随着一些不容忽视的负面影响。正面观点强调了网红在信息传播、社会运动、经济发展和文化多样性等方面的重要作用，这些都是推动社会进步和创新的动力。而反面观点则指出了网红经济带来的信息不准确、诚信缺失、社会压力以及内容同质化等问题，这些挑战可能对个体和社会造成负面影响。

最终回应：

综上所述，网红作为一种新兴社会现象，确实在多个层面上展现出了重要的价值与影响。然而，伴随其而来的问题同样需要引起重视。在享受网红带来的便利与乐趣的同时，社会各界应当共同努力，提升信息传播的质量，强化网红的责任感，推动更健康的网络环境与文化消费。唯有如此，才能确保网红在现代社会中发挥更积极、可持续的作用。

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。

In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

# 初始化 OpenAI 模型
model = ChatOpenAI(model="gpt-4o-mini")

# 创建代码生成模板
code_gen_template = """
You are an expert programmer. Write a {language} implementation of the following:

{requirement}

Only provide the code, no explanations. Make sure the code is complete and can be run directly.
"""

# 为每种语言创建一个链
def create_language_chain(language):
    prompt = ChatPromptTemplate.from_template(code_gen_template)
    return (
        {"language": lambda _: language, "requirement": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

# 创建并行运行的多链
multi_language_chain = RunnableParallel(
    python=create_language_chain("Python"),
    java=create_language_chain("Java"),
    csharp=create_language_chain("C#")
)

# 运行多链并获取结果
requirement = "Write a quicksort algorithm"
results = multi_language_chain.invoke(requirement)

# 打印结果
for language, code in results.items():
    print(f"\n--- {language.upper()} Implementation ---")
    print(code)


--- PYTHON Implementation ---
```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

if __name__ == "__main__":
    array = [3, 6, 8, 10, 1, 2, 1]
    sorted_array = quicksort(array)
    print("Sorted array:", sorted_array)
```

--- JAVA Implementation ---
```java
import java.util.Arrays;

public class QuickSort {
    public static void main(String[] args) {
        int[] array = {3, 6, 8, 10, 1, 2, 1};
        quickSort(array, 0, array.length - 1);
        System.out.println("Sorted array: " + Arrays.toString(array));
    }

    public static void quickSort(int[] array, int low, int high) {
        if (low < high) {
            int partitionIndex = partition(array, low, high);
            quickSort(array, low, partitionIndex - 1);
            quickSort(array

In [9]:
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

if __name__ == "__main__":
    array = [ 1, 3, 3, 2]
    sorted_array = quicksort(array)
    print("Sorted array:", sorted_array)

Sorted array: [1, 2, 3, 3]
